In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import pandas as pd
import requests
import math
import numpy as np

In [75]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [72]:
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()

In [2]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [3]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [4]:
init_notebook_mode(connected=True)

In [5]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [ ]:
crdf = complete_reports_df
cmdf = complete_measurements_df

category = 'A'
state = 'SC'

variables = ['v1', 'v2', 'v3']
states = ['LP', 'SC', 'CO', 'PO', 'BE', 'TA', 'CH', 'PA', 'OR']

state_name_map = {
    'LP': 'La Paz',
    'SC': 'Santa Cruz',
    'CO': 'Cochabamba',
    'PO': 'Potosí',
    'BE': 'Beni',
    'TA': 'Tarija',
    'CH': 'Chuquisaca',
    'PA': 'Pando',
    'OR': 'Oruro', 
}

def get_domain(var):
    if var == 'v1':
        return {"x": [0, .30]}
    if var == 'v2':
        return {"x": [.32, .62]}
    else:
        return {"x": [.64, .94]}
        
def get_visible_list(state):
    state_map = {'LP': 0, 'SC': 1, 'CO': 2, 
                 'PO': 3, 'BE': 4, 'TA': 5,
                 'CH': 5, 'PA': 7, 'OR': 8}
    
    offset = state_map[state]
    
    base_array = [0 for i in range(9)]
    base_array[offset] = 1
    
    big_true = [True for i in range(3)]
    big_false = [False for i in range(3)]
    
    nested_array = [big_true if x == 1 else big_false for x in base_array]
    
    return [x for l in nested_array for x in l]

data = []

for state in states:
    filtered_df = crdf[(crdf.state == state) & (crdf.year == 2017)]
    visible = True if state == 'LP' else False
    for var in variables:
        data.append(dict(
            values=list(filtered_df[var]),
            labels=list(filtered_df.code),
            type='pie',
            hole=.4,
            textposition='inside',
            name=var,
            domain=get_domain(var),
            hoverinfo='label+value+percent+name',
            visible=visible,
        ))
        
var1 = variables_df[variables_df.var_id == 1]
var2 = variables_df[variables_df.var_id == 2]
var3 = variables_df[variables_df.var_id == 3]

v1name = var1.name.iloc[0]
v2name = var2.name.iloc[0]
v3name = var3.name.iloc[0]

v1unit = var1.unit.iloc[0]
v2unit = var2.unit.iloc[0]
v3unit = var3.unit.iloc[0]

updatemenus = [dict(
    active=0,
    xanchor='left',
    yanchor='top',
    direction='up',
    x= .4,
    y= -0.05,
    buttons = [dict(
        label=f'{state_name_map[state]}',
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(state)}, # data modification
            # layout modification
            dict(
#                 title= make_title(ind),
#                 xaxis= make_x_axis(cat),
#                 yaxis= make_y_axis(ind),
#                 shapes= make_shapes(ind),
            ),
        ],
    ) for state in states]
)]


layout = dict(
    updatemenus = updatemenus,
    title = f'<b>V1</b>: {v1name}({v1unit})<br><b>V2</b>: {v2name}({v2unit})<br><b>V3</b>: {v3name}({v3unit})',
    titlefont = dict(size=15),
    annotations = [dict(
        font=dict(size=20),
        showarrow=False,
        text='V1',
        x=0.13,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V2',
        x=0.47,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V3',
        x=0.81,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Fuentes<br>Superficiales',
        x=0.07,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Fuentes<br>Subterráneas',
        x=0.48,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Volumen AP<br>Producido',
        x=0.87,
        y=0,
    )]
)

fig1 = go.Figure(data=data, layout=layout)

In [15]:
#hideme
iplot(fig1)

In [ ]:
categories = ['A', 'B', 'C', 'D']
selected_inds = ['ind23','ind27', 'ind28']

default_cat = 'A'
default_ind = 'ind23'

years = [2014, 2015, 2016, 2017]

data = []

for cat_i, category in enumerate(categories):
    for ind_i, indicator in enumerate(selected_inds):
        visible = True if cat_i == 0 and ind_i == 0 else False
        
        for year_i, year in enumerate(years):
        
            filtered_df = cmdf[(cmdf.category == category) & (cmdf.year == year)]
                    
            current_bar = go.Bar(
                x=filtered_df['code'],
                y=filtered_df[indicator],
                name=str(year),
                text=get_ind_unit(indicator),
                opacity=0.8,
                marker=dict(color=colors[year_i]),
                visible=visible
            )
            
            data.append(current_bar)
            
def get_visible_list(category, indicator):
    category_map = dict(A=0, B=1, C=2, D=3)
    indicator_map = dict(ind23=0, ind27=1, ind28=2)
    
    cat_offset = category_map[category]
    ind_offset = indicator_map[indicator]
    
    offset = (cat_offset * 3) + ind_offset
    
    base_array = [0 for i in range(12)]
    base_array[offset] = 1
    
    big_true = [True for i in range(4)]
    big_false = [False for i in range(4)]
    
    nested_array = [big_true if x == 1 else big_false for x in base_array]
    
    return [x for l in nested_array for x in l]

def make_x_axis(category):
    return dict(
        title=f'EPSAs Categoría {category}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_y_axis(ind_code):
    return dict(
        title=f'Unidad: {get_ind_unit(ind_code)}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_title(ind_code):
    ind_name = get_ind_name(ind_code)
    ind_unit = get_ind_unit(ind_code)
    ind_param = '(Parámetro óptimo: 65-75%)' if ind_code == 'ind23' else ''
    return f'Indicador {ind_code[3:]}: {ind_name} ({ind_unit}) {ind_param}'
    

def make_shapes(ind_code):
    
    special_shapes = [dict(
        type= 'rect',
        xref= 'paper',
        yref= 'y',
        x0= 0,
        y0= 65,
        x1=1,
        y1= 75,
        fillcolor= colors[0],
        opacity= 0.2,
        line= {'width': 0,}
    )]
    
    return special_shapes if ind_code == 'ind23' else []

    
drop_x_map = dict(
    ind23=0,
    ind27=0.3,
    ind28=0.6,
)

updatemenus = [dict(
    active=0,
    xanchor='left',
    yanchor='top',
    direction='up',
    x= drop_x_map[ind],
    y= -0.25,
    buttons = [dict(
        label=f'Indicador {ind[3:]} - categoría {cat}',
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(cat, ind)}, # data modification
            # layout modification
            dict(
                title= make_title(ind),
                xaxis= make_x_axis(cat),
                yaxis= make_y_axis(ind),
                shapes= make_shapes(ind),
            ),
        ],
    ) for cat in categories]
) for ind in selected_inds]

layout = go.Layout(
    title= make_title(default_ind),
    xaxis= make_x_axis(default_cat),
    yaxis= make_y_axis(default_ind),
    shapes=make_shapes(default_ind),
    updatemenus = updatemenus,
)

fig2 = go.Figure(data=data, layout=layout)

In [8]:
iplot(fig2)

In [10]:
import json

supply_r = requests.get('https://peridash.ml/api/supply-areas/')

supply_json = json.loads(supply_r.text)

epsa_jsons = []
for feature in supply_json['features']:
    epsa_json = dict(
        type='FeatureCollection',
        features=[feature]
    )
    epsa_jsons.append(epsa_json)
    
with open('points.json', encoding='utf8') as f:
    points_json = json.load(f)

In [ ]:
df = complete_measurements_df

val_year = 2017

min_value = min(list(df[df.year == val_year].ind8))

min_ranges = [0.0, 0.35, 0.5, 0.6, 0.7]
max_ranges = [0.35, 0.5, 0.6, 0.7, 1.1]
min_colors = [[220,220,220],[106,137,247],[90,120,245],[70,100,245],[40,60,190]]
max_colors = [[106,137,247],[90,120,245],[70,100,245],[40,60,190],[5,10,172]]

val_label = 'Indicador 8'
val_label2 = 'Indicador 9'
val_unit = '%'


def interpolate(c1, c2, f):
    r = c1[0] + (c2[0]-c1[0]) * f
    g = c1[1] + (c2[1]-c1[1]) * f
    b = c1[2] + (c2[2]-c1[2]) * f
    return [r,g,b]

def get_layer_code(layer_index):
    code = epsa_jsons[layer_index]['features'][0]['properties']['code']
    code = code.split('-')[0]
    return code

def get_layer_color(epsa_code, min_value):
    if epsa_code in list(df.code):
        percentage = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(percentage):
            percentage = 0 
    else:
        percentage = 0
        
    factor = (percentage - min_value)/(100 - min_value) # Biased factor for more contrast
    
    color1 = min_colors[0]
    color2 = min_colors[1]
    real_factor = 0.0
    for range_min, range_max, c_min, c_max in zip(min_ranges, max_ranges, min_colors, max_colors):
        if range_min <= factor < range_max:
            color1 = c_min
            color2 = c_max
            real_factor = (factor - range_min) / (range_max - range_min)
    
    c = interpolate(color1, color2, real_factor)
    color_string = f'rgb({str(c[0])[:6]},{str(c[1])[:6]},{str(c[2])[:6]})'
    return(color_string)

def get_display_value(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind8.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

def get_display_value2(epsa_code):
    if epsa_code in list(df.code):
        val = df[(df.code==epsa_code)&(df.year==val_year)].ind9.iloc[0]
        if np.isnan(val):
            val = 0.0
    else:
        val = 0.0
    return val

layers = [
    dict(
        sourcetype = 'geojson',
        source = epsa_jsons[k],
#         below = "water",
        type = 'fill',   
        color = get_layer_color(get_layer_code(k), min_value),
        opacity = 0.8
    ) for k in range(len(epsa_jsons))
]

p_lats = []
p_lons = []
p_texts = []

for code in list(points_json.keys()):
    epsa_p = points_json[code]
    n = len(epsa_p['lats'])
    p_lats += epsa_p['lats']
    p_lons += epsa_p['lons']
    
    clean_code = code.split('-')[0]
    value = get_display_value(clean_code)
    value2= get_display_value2(clean_code)
    display_text = f'{clean_code}<br>{val_label}: {value} ({val_unit})<br>{val_label2}: {value2} ({val_unit})'
    p_texts += [display_text for k in range(n)]
    
scatter_dict = dict(
    type = 'scattermapbox',
    mode = 'markers',
    
    lat = p_lats, 
    lon = p_lons,
    text = p_texts,
    
    marker = dict(
        size = 50,
        opacity = 0,
        color= 'rgb(37,52,148)'
    ),
    showlegend = False,
    hoverinfo = 'text',
)

mapbox_access_token = 'pk.eyJ1Ijoic2VyZ2lvLWNodW1hY2Vyby1maSIsImEiOiJjamswOTUzeHkwMDk0M3dvNnJoeTByZGlpIn0.3mmjpLwDrIUcdJTowlCd1A'

layout = dict(
    title = 'Indicadores de Cobertura',
    font = dict(family='Balto'),
    autosize = False,
    width = 1200,
    height = 700,
    hovermode = 'closest',

    mapbox = dict(
        accesstoken = mapbox_access_token,
        layers = layers,
        bearing = 0,
        center = dict( 
            lat = -17.610907366555434, 
            lon = -63.13396632812757,
        ),
        pitch = 0,
        zoom = 8,
        style = 'light'
    ) 
)

fig3 = dict(data=[scatter_dict], layout=layout)

import plotly.plotly as py

In [13]:
py.iplot(fig3, filename='mapa-coberturas-new')

In [76]:
num_indicators = 32
cmdf = complete_measurements_df
ind_list = [f'ind{i+1}' for i in range(num_indicators)]

def get_data_rating(epsa_code, year):
    filtered_df = cmdf[(cmdf.year == year) & (cmdf.code == epsa_code)][ind_list]
    
    if len(filtered_df) > 0:
        values = list(filtered_df.iloc[0])
    else:
        values = [math.nan] * len(list(filtered_df))
        
    num_reported = sum([0 if math.isnan(val) else 1 for val in values])
    return num_reported / num_indicators * 100

selected_category = 'B'
cmdf = complete_measurements_df
filtered_df = cmdf[cmdf.category == selected_category]
epsa_codes = list(filtered_df.code.unique())
years = list(filtered_df.sort_values('year').year.unique())

scat1 = go.Scatter(
    x= [2014] * len(epsa_codes),
    y= list(filtered_df[filtered_df.year == 2014].ind1),
    text= epsa_codes,
    mode='markers+text',
    showlegend=False,
)

data = [scat1]

for epsa_code in epsa_codes:
    line_trace = go.Scatter(
        x= years,
#         y= [get_data_rating(epsa_code, year) for year in years],
        y= list(filtered_df[filtered_df.code == epsa_code].sort_values('year').ind1),
        mode='lines',
        name=epsa_code,
#         hoverinfo='none',
    )
    data.append(line_trace)
    
layout = dict(
    xaxis= dict(
        title='Años',
        range=[2013.8, 2017.2],
        autorange= False,
        dtick=1,
    ),
    yaxis= dict(
        title='Porcentaje de variables reportadas (%)',
#         dtick=1,
#         range=[50,100],
#         autorange=False,
    ),
    title= f'Ranking EPSAS - Porcentaje de variables reportadas<br>Categoría {selected_category}',
    updatemenus= [dict(
        x=0,
        y=0,
        type= 'buttons',
        buttons=[dict(
            label='Animar',
            method='animate',
            args= [None],
        )],
    )],
    hovermode='closest',
)

colors = [
    '#1f77b4', '#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

colors = colors * 3

frames = [{'data': [go.Scatter(
    x=[year]*len(epsa_codes),
#     y=[get_data_rating(epsa_code, year) for epsa_code in epsa_codes],
    y=list(filtered_df[filtered_df.year == year].ind1),
    text=epsa_codes,
    textposition='top center',
    mode='markers',
    marker=dict(symbol='circle', size=20, color=colors[1:]),
)]} for year in years]

fig4b = go.Figure(data=data, layout=layout, frames=frames)

In [68]:
iplot(fig4a)

In [77]:
iplot(fig4b)

In [70]:
iplot(fig4c)

In [71]:
iplot(fig4d)

In [59]:
# filtered_df = cmdf[(cmdf.year == years[0]) & (cmdf.code == epsa_codes[0])][ind_list]

# len(list(filtered_df))

# list(cmdf[(cmdf.year == years[0]) & (cmdf.code == epsa_codes[0])][['ind1']].iloc(0))

math.nan

nan